Importamos dependencias

In [1]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv

load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew, Process

Definimos el modelo

In [2]:
from crewai import LLM

diagnostic_llm = LLM(
    model="openai/gpt-4o-mini", # call model by provider/model_name
    temperature=0.8,
    max_tokens=1000,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)
suggestion_llm = LLM(
    model="openai/gpt-4o-mini", # call model by provider/model_name
    temperature=0.8,
    max_tokens=1500,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

Carga de instrucciones

In [3]:
files = {
    'agents': '../../app/config/hair_diagno_config/agents.yaml',
    'tasks': '../../app/config/hair_diagno_config/tasks.yaml'
}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents']
tasks_config = configs['tasks']

Pydantic model for the diagnostic task y color sugestion

In [4]:
from typing import List, Optional
from pydantic import BaseModel, Field, field_validator
from enum import Enum

class ToneTemperature(str, Enum):
    WARM = "cálido"
    COLD = "frío"
    NEUTRAL = "neutro"

class DamageLevel(str, Enum):
    LIGHT = "ligero"
    MODERATE = "moderado"
    SEVERE = "severo"

class Porosity(str, Enum):
    LOW = "baja"
    MEDIUM = "media"
    HIGH = "alta"

class HairCharacteristics(BaseModel):
    porosity: Porosity = Field(..., description="Nivel de porosidad del cabello")
    thickness: str = Field(..., description="Grosor del cabello (fino, medio, grueso)")
    density: str = Field(..., description="Densidad del cabello (baja, media, alta)")
    texture: str = Field(..., description="Textura del cabello (liso, ondulado, rizado, etc.)")
    damage: DamageLevel = Field(..., description="Nivel de daño del cabello")

class HairDiagnostic(BaseModel):
    base_tone_height: int = Field(
        ..., 
        description="Altura del tono base en escala internacional (1-10)",
        ge=1, # greater than or equal
        le=10 # less than or equal
    )
    gray_hair_percentage: float = Field(
        ..., 
        description="Porcentaje aproximado de canas presentes",
        ge=0, 
        le=100
    )
    mid_ends_color_description: str = Field(
        ...,
        description="Descripción detallada del color en medios y puntas"
    )
    tone_temperature: ToneTemperature = Field(
        ...,
        description="Temperatura del tono actual (cálido, frío, neutro)"
    )
    tone_description: str = Field(
        ...,
        description="Descripción técnica completa del tono con matices específicos"
    )
    hair_characteristics: HairCharacteristics = Field(
        ...,
        description="Características estructurales del cabello"
    )
    recommendations: List[str] = Field(
        default=[],
        description="Recomendaciones basadas en el diagnóstico"
    )
 
# Color sugestion schema

class ColorationType(str, Enum):
    PERMANENT = "coloración de oxidación permanente"
    SEMIPERMANENT = "tono sobre tono"
    DIRECT = "coloración directa"
    FANTASY = "coloración fantasía"

class MixtureRatio(BaseModel):
    color_code: str = Field(..., description="Código o número del color (ej: 7, 7.32)")
    quantity: str = Field(..., description="Cantidad del color (ej: 1/2 tubo, 30ml)")

class ColorSuggestion(BaseModel):
    # Campos existentes
    mixture_name: str = Field(...)
    mixture_type: str = Field(...)
    coloration_name: str = Field(...)
    coloration_type: ColorationType = Field(...)
    color_mixture_description: str = Field(...)
    components: List[MixtureRatio] = Field(...)
    developer_volume: str = Field(...)
    application_time: str = Field(...)
    
    # Campos ampliados para recomendaciones detalladas
    special_considerations: List[str] = Field(
        default=[],
        description="Consideraciones especiales basadas en características del cabello"
    )
    aftercare_recommendations: List[str] = Field(
        default=[],
        description="Recomendaciones de cuidado posterior a la aplicación"
    )
    maintenance_plan: List[str] = Field(
        default=[],
        description="Plan de mantenimiento personalizado según características"
    )
    expected_result: str = Field(...)
    detailed_explanation: str = Field(
        default="",
        description="Explicación detallada del razonamiento detrás de la formulación"
    )

IMAGE ANALYZER TOOL

In [5]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
import base64
import os
from typing import Optional, Type
from PIL import Image
import io
import anthropic

# 1. Definimos un esquema compatible con CrewAI v0.22+
class AnalyzeImageArgs(BaseModel):
    image_path: str = Field(..., description="Ruta al archivo de imagen a analizar")

# 2. Creamos una herramienta compatible con BaseTool
class AnalyzeImageTool(BaseTool):
    name: str = "analyze_image"
    description: str = "Analiza una imagen de cabello utilizando Anthropic Claude Vision API"
    args_schema: Type[BaseModel] = AnalyzeImageArgs
    
    def _run(self, image_path: str) -> str:
        try:       
            # Verificar si la imagen existe
            image_path = os.path.abspath(os.path.expanduser(image_path))
            
            if not os.path.exists(image_path):
                return f"Error: La imagen no existe en la ruta: {image_path}"            
            # Detectar si es WEBP y convertir a JPEG si es necesario
            base64_image = ""
            try:
                with Image.open(image_path) as img:                    
                    # Si es WEBP u otro formato no compatible directamente, convertir a JPEG
                    buffer = io.BytesIO()
                    img = img.convert('RGB')  # Elimina canal alfa si existe
                    img.save(buffer, format='JPEG')
                    buffer.seek(0)
                    image_data = buffer.read()
                    base64_image = base64.b64encode(image_data).decode("utf-8")
            except Exception as img_error:
                return f"Error al procesar la imagen: {str(img_error)}"
            
            if len(base64_image) == 0:
                return "Error: La imagen no pudo ser codificada correctamente"
            
            prompt = """Analizar detalladamente la imagen del cabello proporcionada y generar un diagnóstico profesional completo. El análisis debe:
                - Determinar la altura del tono base utilizando la escala internacional de colorimetría (1-10)
                - Calcular el porcentaje aproximado de canas presentes en los siguientes porcentajes (0% 25% 50% 75% 100%)
                - Identificar y describir el color en medios y puntas
                - Proporcionar una descripción técnica del tono actual (cálido, frío, neutro)
                - Analizar las características del cabello (porosity, thickness, density, texture, damage)
                
                Si no puedes procesar la imagen claramente o no contiene cabello visible, NO inventes ningún diagnóstico,
                simplemente indica que no puedes realizar el análisis."""

            
            try:
                client = anthropic.Anthropic()
                
                response = client.messages.create(
                    model="claude-3-5-haiku-20241022", # claude-3-7-sonnet-20250219
                    max_tokens=1000,
                    messages=[
                        {
                            "role": "user",
                            "content": [
                                {
                                    "type": "image",
                                    "source": {
                                        "type": "base64",
                                        "media_type": "image/jpeg",
                                        "data": base64_image,
                                    },
                                },
                                {
                                    "type": "text",
                                    "text": prompt
                                }
                            ],
                        }
                    ],
                )
                return response.content[0].text
            except Exception as api_error:
                return f"Error al enviar la imagen a la API: {str(api_error)}"
            
        except Exception as e:
            return f"Error al analizar la imagen: {str(e)}"

####  UPLOAD TOOL

In [6]:
# Implementar CloudflareUploadTool directamente en el notebook
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
import boto3
from botocore.client import Config
import uuid
from typing import Optional, Type
from PIL import Image
import io
import os

class CloudflareUploadArgs(BaseModel):
    image_path: str = Field(..., description="Ruta local al archivo de imagen a subir")
    file_name: Optional[str] = Field(None, description="Nombre personalizado para el archivo en Cloudflare (opcional)")
    
# class CloudflareUploadTool(BaseTool):
#     name: str = "upload_to_cloudflare"
#     description: str = "Sube una imagen a un bucket de Cloudflare R2 y devuelve la URL pública"
#     args_schema: Type[BaseModel] = CloudflareUploadArgs
    
    def _run(self, image_path: str, file_name: Optional[str] = None) -> str:
        try:
            # Verificación de variables de entorno
            required_vars = ['CLOUDFLARE_ACCESS_KEY_ID', 'CLOUDFLARE_SECRET_ACCESS_KEY', 
                            'CLOUDFLARE_BUCKET_NAME', 'CLOUDFLARE_DOMAIN']
            missing_vars = [var for var in required_vars if not os.environ.get(var)]
            
            if missing_vars:
                return f"ERROR: Faltan variables de entorno necesarias: {', '.join(missing_vars)}"
            
            # Mostrar configuración
            bucket_name = os.environ['CLOUDFLARE_BUCKET_NAME']
            print(f"CONFIG: Bucket={bucket_name}")
            print(f"CONFIG: Endpoint={os.environ.get('CLOUDFLARE_DOMAIN')}")
            
            # Verificar si la imagen existe
            image_path = os.path.abspath(os.path.expanduser(image_path))
            if not os.path.exists(image_path):
                return f"Error: La imagen no existe en la ruta: {image_path}"
            
            # Leer la imagen
            try:
                with open(image_path, 'rb') as img_file:
                    image_data = img_file.read()
                    print(f"Imagen leída correctamente - {len(image_data)} bytes")
            except Exception as img_error:
                return f"Error al leer la imagen: {str(img_error)}"
            
            # Configurar endpoint
            endpoint_url = os.environ.get('CLOUDFLARE_DOMAIN', '')
            if not endpoint_url.startswith('https://'):
                endpoint_url = f"https://{endpoint_url}"
            
            # Configurar cliente S3 - NO usamos list_buckets
            try:
                s3 = boto3.client('s3',
                    endpoint_url = endpoint_url,
                    aws_access_key_id = os.environ.get('CLOUDFLARE_ACCESS_KEY_ID'),
                    aws_secret_access_key = os.environ.get('CLOUDFLARE_SECRET_ACCESS_KEY'),
                    config = Config(signature_version = 's3v4'),
                )
                
                # En lugar de list_buckets, verificamos directamente si podemos acceder al bucket
                if not file_name:
                    file_name = f"image_{uuid.uuid4()}.jpg"
                
                object_key = file_name
                print(f"Subiendo '{object_key}' al bucket '{bucket_name}'...")
                
                try:
                    # Subir el archivo con try/except específico
                    response = s3.put_object(
                        Bucket=bucket_name,
                        Key=object_key,
                        Body=image_data,
                        ContentType='image/jpeg'
                    )
                    
                    if 'ResponseMetadata' in response and response['ResponseMetadata'].get('HTTPStatusCode') == 200:
                        print(f"Archivo subido correctamente: {response}")
                        
                        # Generar URL pública correcta
                        # Opción 1: Estándar para buckets públicos
                        public_url = f"https://{bucket_name}.r2.dev/{object_key}" 
                        
                        # Opción 2: Usando dominio personalizado si existe
                        if "CLOUDFLARE_PUBLIC_URL" in os.environ:
                            public_url = f"{os.environ.get('CLOUDFLARE_PUBLIC_URL')}/{object_key}"
                            
                        return f"✅ Imagen subida exitosamente.\nURL: {public_url}"
                    else:
                        return f"Error: Respuesta inesperada: {response}"
                
                except Exception as upload_error:
                    import traceback
                    return f"ERROR al subir archivo: {str(upload_error)}\n{traceback.format_exc()}"
                
            except Exception as s3_error:
                import traceback
                return f"ERROR al configurar cliente S3: {str(s3_error)}\n{traceback.format_exc()}"
            
        except Exception as e:
            import traceback
            return f"ERROR general: {str(e)}\n{traceback.format_exc()}"

Agnents & Crews

In [7]:

# Creating Agents


hair_diagno = Agent(
  config=agents_config['hair_diagno'],  
  llm=diagnostic_llm,
  multimodal=True,
  verbose=True
)

color_suggestion = Agent(
  config=agents_config['color_suggestion'],  
  llm=suggestion_llm,
  multimodal=True,
  verbose=True
)

hair_diagno_generation = Task(
  config=tasks_config['hair_diagno_generation'],
  agent=hair_diagno,
  output_pydantic= HairDiagnostic
)

color_suggestion_generation = Task(
  config=tasks_config['color_suggestion_generation'],
  agent=color_suggestion,
  output_pydantic= ColorSuggestion,
)

crew= Crew(
  agents=[hair_diagno, color_suggestion],
  tasks=[hair_diagno_generation, color_suggestion_generation],
  verbose=True,
   process=Process.sequential
)

# Usamos una ruta absoluta para asegurarnos de que se encuentra la imagen
# Alternativamente, puedes probar con una imagen que sepas que existe

image_path = 'https://pub-b213a1fb448445b2a1bd745a11fba172.r2.dev/image_f3df33be-15a0-42f7-a17d-e8185b8728b8.jpg'

# # Añade esta celda para probar directamente
# tool = CloudflareUploadTool()
# result = tool._run(image_path)
# print(result)

# Para depuración, verifica la existencia de la imagen antes de ejecutar
print(f"¿La imagen existe? {os.path.exists(image_path)}")

result = crew.kickoff(inputs={'image_path': image_path})

# Añade esto después de obtener los resultados
if isinstance(result, list) and len(result) >= 2:
    diagnostico = result[0]
    formulacion = result[1]
    
    print("\n==== RESULTADO FINAL DEL ANÁLISIS Y RECOMENDACIÓN ====\n")
    
    # Puedes formatear la salida o simplemente imprimir la respuesta completa
    print(formulacion)


¿La imagen existe? False
# Agent: Consultor de Diagnóstico Capilar
## Task: 1. PRIMERO: analiza la imagen https://pub-b213a1fb448445b2a1bd745a11fba172.r2.dev/image_f3df33be-15a0-42f7-a17d-e8185b8728b8.jpg.
3. TERCERO: Estructura el diagnóstico en el formato indicado en el expected_output, e incluye la URL pública retornada por la herramienta de upload.
Es OBLIGATORIO que uses AMBAS herramientas en este orden.


# Agent: Consultor de Diagnóstico Capilar
## Final Answer: 
{
  "base_tone_height": 7,
  "gray_hair_percentage": 25,
  "mid_ends_color_description": "El color en medios y puntas muestra un tono rubio claro con reflejos dorados, presentando una ligera variación de color que indica una posible decoloración o tratamiento previo.",
  "tone_temperature": "Cálido",
  "tone_description": "Familia de color rubio, con subtonos dorados y matices claros, que dan una apariencia luminosa.",
  "hair_characteristics": {
    "porosity": "Media",
    "thickness": "Fino",
    "density": "Bajo",
 